# Create Spark Datatable
This notebook collects data from various inputs, including the underlying claims to build a panel format Spark table.

## Run Install Requirements

In [0]:
%run "../Requirements"

## Run Reference Lists
Note: OMS lookup table accessed via CCW Snowflake

In [0]:
%run "../data/Reference Tables/reference_lists"

## Run Utilities

In [0]:
%run "../data/utils"

## Run Reference Tables

In [0]:
%run "../data/Reference Tables/Import Opioid Reference"

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.functions import year, countDistinct
from pyspark.sql.types import ArrayType, DoubleType, IntegerType, ShortType, FloatType, ByteType

## Build Cohort Dataframe

### Main Query

In [0]:
# Main Query
# Cohort: 65+ Continuously-enrolled FFS ABD benes (non-hospice, non-dual) in 2020
# Target: non-elective opioid-related adverse event ICD-10 code in hospital/ED between Nov-Dec 2020

cohort_df = spark.sql(f"""
SELECT 
a.bene_id,
a.clm_id,
a.line_num,
a.srvc_dlvrd, 
a.first_srvc_dt, 
a.prvdr_spclty,
a.prvdr_id,
a.dgns_cd1,
a.dgns_cd2,
a.dgns_cd3,
a.dgns_cd4,
a.dgns_cd5,
a.dgns_cd6,
a.dgns_cd7,
a.dgns_cd8,
a.dgns_cd9,
a.dgns_cd10,
a.dgns_cd11,
a.dgns_cd12,
a.pymt_sys,
a.srvc_vol,
a.clm_days,

CASE WHEN
  (DGNS_CD1 in ({FULL_Opx_Dx_list}) OR
  DGNS_CD2 in ({FULL_Opx_Dx_list}) OR
  DGNS_CD3 in ({FULL_Opx_Dx_list}) OR
  DGNS_CD4 in ({FULL_Opx_Dx_list}) OR
  DGNS_CD5 in ({FULL_Opx_Dx_list}) OR
  DGNS_CD6 in ({FULL_Opx_Dx_list}) OR
  DGNS_CD7 in ({FULL_Opx_Dx_list}) OR
  DGNS_CD8 in ({FULL_Opx_Dx_list}) OR
  DGNS_CD9 in ({FULL_Opx_Dx_list}) OR
  DGNS_CD10 in ({FULL_Opx_Dx_list}) OR
  DGNS_CD11 in ({FULL_Opx_Dx_list}) OR
  DGNS_CD12 in ({FULL_Opx_Dx_list}))
  AND ER_count > 0 
  AND first_srvc_dt BETWEEN '2020-11-01' and '2020-12-31' -- target period
  AND CLM_IP_ADMSN_TYPE_CD not in ('3') -- non-elective
THEN 1 
ELSE 0 
END AS target,
count(*) as rows_count

FROM eldb.rt_eldb_pt_abd_2017_2022_service_level_100pct_binned a

INNER JOIN (SELECT bene_id,  
CASE WHEN BUYIN_desc in('Part A and Part B','Part A state buy-in','Part B state buy-in','Part A and Part B state buy-in') AND OP_MDCD_monthly = 'NONDUAL' AND PTD_enrlmt = 'Enrolled in PTD for the month' AND HMOIND_desc = 'Not a member of an HMO'
THEN row_number() OVER (PARTITION BY bene_id ORDER BY month_begin) END AS ABD_NONDUAL_months

FROM eldb.longitudinal_eldb_bene 
WHERE year(month_begin) = 2020) b on a.bene_id = b.bene_id

LEFT JOIN (SELECT clm_id, 
COUNT(CASE WHEN REV_CNTR_CD in ('0450', '0451', '0452', '0456', '0459','0981') 
THEN 1 
ELSE 0 END) AS ER_count

FROM extracts.gvclm18_pta_revenue_2020
GROUP BY 1) c on a.clm_id = c.clm_id

LEFT JOIN (SELECT clm_id, 
CLM_IP_ADMSN_TYPE_CD

FROM extracts.gvclm18_pta_claim_2020
) d on a.clm_id = d.clm_id

ANTI JOIN (SELECT bene_id
FROM extracts.gvclm18_pta_revenue_2020
WHERE SRVC_2 = 'HOS' AND REV_CNTR_CD in ('0651', '0652', '0655','0656') 
) c2 on a.bene_id = c2.bene_id

WHERE year(first_srvc_dt) = 2020 and (ABD_NONDUAL_months=12 or (ABD_NONDUAL_months = 11 and bene_death_dt between '2020-12-01' and '2020-12-31') or (ABD_NONDUAL_months = 10 and bene_death_dt between '2020-11-01' and '2020-11-30')) and bene_age >=65 and (bene_death_dt is null or bene_death_dt >= '2020-11-01')
GROUP BY 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23 -- making sure there are no duplicates, especially in Part D events

ORDER BY bene_id, first_srvc_dt
""")

BEGIN_YR,END_YR,YEAR_GAP,FDB_NDC,FDB_BN,FDB_GNN,FDB_STR,FDB_FORM,FDB_RT,FDB_GTC,FDB_TC,FDB_CL,FDB_MFG,FDB_MONO,FDB_DEA,FDB_BG_IND,FDB_IND,FDB_ETC_ID,FDB_ETC_NAME,OMS_OPIOID_FLAG,OMS_MAT_FLAG,OMS_METHADONE_FLAG,OMS_BENZO_FLAG,OMS_NALOXONE_FLAG,OMS_MME_CONVERSION_FACTOR,OMS_OPIOID_STR,CDC_OPIOID_LA_SA_ACTING_FLAG,OTC_MDCD,ANTIBIOTIC_FLAG,ANTIPSYCH_FLAG
2019.0,2020.0,1.0,64764089099,CONTRAVE,NALTREXONE HCL/BUPROPION HCL,8 MG-90 MG,"TABLET, EXTENDED RELEASE",ORAL,8.0,13.0,F,TAKEDA/OREXIGEN,"USES: This medication is used with a doctor-approved exercise, behavior change, and reduced-calorie diet program to help you lose weight. It is used by certain overweight people, such as those who are obese or have weight-related medical problems. Losing weight and keeping it off can lessen the many health risks that come with obesity, including heart disease, diabetes, high blood pressure, and a shorter life. Naltrexone belongs to a class of drugs known as opiate antagonists, and bupropion is an antidepressant that may help restore the balance of certain natural chemicals (neurotransmitters) in your brain. These two medications work together on separate parts of the brain to reduce appetite and how much you eat. Most opiates should not be used for 7 to 10 days before starting naltrexone, but some opiate drugs (such as methadone) should not be used for 10 to 14 days before starting naltrexone. Ask your doctor or pharmacist for more details. Other forms of naltrexone or bupropion are used to treat a variety of conditions including depression, other mental/mood disorders, smoking cessation, addiction to opioid medication, and alcohol abuse. This combination product is not approved for these other uses. Discuss the risks and benefits of this medication, as well as other ways to lose weight, with your doctor.",0,B,Y,6498.0,Anti-Obesity-Opioid Antag/Norepinephrine and Dopamine Reuptake Inhibit,N,N,N,N,N,null,null,null,N,N,N
2019.0,2020.0,1.0,65757030202,VIVITROL,NALTREXONE MICROSPHERES,380 MG,"SUSPENSION, EXTENDED RELEASE, RECON (EA)",INTRAMUSC,99.0,87.0,F,ALKERMES INC,"USES: This medication is used to treat alcohol abuse. It is used only in people who have been able to stop drinking for some time before starting treatment with naltrexone. You should not be drinking when you start naltrexone. It can help people drink less alcohol or stop drinking altogether. Naltrexone works in the brain to decrease the desire to drink. It does not work like some other treatments for alcohol abuse (e.g., disulfiram). It will not make you sick when taken with alcohol. This medication is also used to prevent relapse to opioid abuse, after opioid detoxification. It works by blocking the action of opioids. This medication must not be used in people currently taking opiates, including methadone. Doing so can cause sudden withdrawal symptoms. Naltrexone belongs to a class of drugs known as opiate antagonists. It is used as part of a complete treatment program for alcohol or opioid abuse (e.g., counseling, 12-step program, lifestyle changes).",0,B,Y,5952.0,Alcohol Abstinence Therapy - Opioid Receptor Antagonist-Type,N,N,N,N,N,null,null,null,N,N,N
2019.0,2020.0,1.0,65757030001,VIVITROL,NALTREXONE MICROSPHERES,380 MG,"SUSPENSION, EXTENDED RELEASE, RECON (EA)",INTRAMUSC,99.0,87.0,F,ALKERMES INC,"USES: This medication is used to treat alcohol abuse. It is used only in people who have been able to stop drinking for some time before starting treatment with naltrexone. You should not be drinking when you start naltrexone. It can help people drink less alcohol or stop drinking altogether. Naltrexone works in the brain to decrease the desire to drink. It does not work like some other treatments for alcohol abuse (e.g., disulfiram). It will not make you sick when taken with alcohol. This medication is also used to prevent relapse to opioid abuse, after opioid detoxification. It works by blocking the action of opioids. This medication must not be used in people currently taking opiat

your 'sf_options()' function is ready to use...

### Join reference table

In [0]:

# Join drug ref table from reference table
pre_joined_df = (cohort_df.join(drug_ref_df, [drug_ref_df.FDB_NDC == cohort_df.srvc_dlvrd, year(cohort_df.first_srvc_dt).between(drug_ref_df.BEGIN_YR, drug_ref_df.END_YR)], 'left'))

# Select distinct benes that have a non-MAT opioid during October
opioid_group = pre_joined_df.filter("OMS_OPIOID_FLAG = 'Y' AND OMS_NALOXONE_FLAG = 'N' AND OMS_MAT_FLAG = 'N' AND FDB_IND = 'Y' AND first_srvc_dt between '2020-10-01' and '2020-10-31' AND FDB_GNN != 'AMLODIPINE BESYLATE'").select('bene_id').distinct()
# Select only benes in cohort table that have a non-MAT opioid during October
joined_df = pre_joined_df.join(opioid_group, 'bene_id', 'inner')

# Get cum sum of distinct Opx prescribers (IDs)
window_spec = (Window.partitionBy('bene_id').orderBy('first_srvc_dt')
             .rowsBetween(Window.unboundedPreceding, 0))
opioid_group_rxers =  (pre_joined_df
                       .filter("OMS_OPIOID_FLAG = 'Y' AND OMS_NALOXONE_FLAG = 'N' AND OMS_MAT_FLAG = 'N' AND FDB_IND = 'Y' AND FDB_GNN != 'AMLODIPINE BESYLATE'")
                       .withColumn("cum_distinct_opx_prescribers", 
                                   F.size(F.collect_set('PRVDR_ID').over(window_spec)))
                                           )

# Add data from cumulative prescribers to main table            
joined_df = (joined_df.join(opioid_group_rxers
                   .select('bene_id', 'clm_id', 'first_srvc_dt', 'cum_distinct_opx_prescribers'), 
                           ['bene_id', 'clm_id', 'first_srvc_dt'], 
                           'left'))


In [0]:
# Cache table
joined_df.cache().count()

Out[10]: 110642033

### View Details

In [0]:
# Total benes
display(joined_df.agg(countDistinct('bene_id')))
# Opioid hosp/ED visit benes
display(joined_df.filter("target = 1").agg(countDistinct('bene_id')))

count(bene_id)
654346


count(bene_id)
1450


BEGIN_YR,END_YR,YEAR_GAP,FDB_NDC,FDB_BN,FDB_GNN,FDB_STR,FDB_FORM,FDB_RT,FDB_GTC,FDB_TC,FDB_CL,FDB_MFG,FDB_MONO,FDB_DEA,FDB_BG_IND,FDB_IND,FDB_ETC_ID,FDB_ETC_NAME,OMS_OPIOID_FLAG,OMS_MAT_FLAG,OMS_METHADONE_FLAG,OMS_BENZO_FLAG,OMS_NALOXONE_FLAG,OMS_MME_CONVERSION_FACTOR,OMS_OPIOID_STR,CDC_OPIOID_LA_SA_ACTING_FLAG,OTC_MDCD,ANTIBIOTIC_FLAG,ANTIPSYCH_FLAG
2019.0,2020.0,1.0,64764089099,CONTRAVE,NALTREXONE HCL/BUPROPION HCL,8 MG-90 MG,"TABLET, EXTENDED RELEASE",ORAL,8.0,13.0,F,TAKEDA/OREXIGEN,"USES: This medication is used with a doctor-approved exercise, behavior change, and reduced-calorie diet program to help you lose weight. It is used by certain overweight people, such as those who are obese or have weight-related medical problems. Losing weight and keeping it off can lessen the many health risks that come with obesity, including heart disease, diabetes, high blood pressure, and a shorter life. Naltrexone belongs to a class of drugs known as opiate antagonists, and bupropion is an antidepressant that may help restore the balance of certain natural chemicals (neurotransmitters) in your brain. These two medications work together on separate parts of the brain to reduce appetite and how much you eat. Most opiates should not be used for 7 to 10 days before starting naltrexone, but some opiate drugs (such as methadone) should not be used for 10 to 14 days before starting naltrexone. Ask your doctor or pharmacist for more details. Other forms of naltrexone or bupropion are used to treat a variety of conditions including depression, other mental/mood disorders, smoking cessation, addiction to opioid medication, and alcohol abuse. This combination product is not approved for these other uses. Discuss the risks and benefits of this medication, as well as other ways to lose weight, with your doctor.",0,B,Y,6498.0,Anti-Obesity-Opioid Antag/Norepinephrine and Dopamine Reuptake Inhibit,N,N,N,N,N,null,null,null,N,N,N
2019.0,2020.0,1.0,65757030202,VIVITROL,NALTREXONE MICROSPHERES,380 MG,"SUSPENSION, EXTENDED RELEASE, RECON (EA)",INTRAMUSC,99.0,87.0,F,ALKERMES INC,"USES: This medication is used to treat alcohol abuse. It is used only in people who have been able to stop drinking for some time before starting treatment with naltrexone. You should not be drinking when you start naltrexone. It can help people drink less alcohol or stop drinking altogether. Naltrexone works in the brain to decrease the desire to drink. It does not work like some other treatments for alcohol abuse (e.g., disulfiram). It will not make you sick when taken with alcohol. This medication is also used to prevent relapse to opioid abuse, after opioid detoxification. It works by blocking the action of opioids. This medication must not be used in people currently taking opiates, including methadone. Doing so can cause sudden withdrawal symptoms. Naltrexone belongs to a class of drugs known as opiate antagonists. It is used as part of a complete treatment program for alcohol or opioid abuse (e.g., counseling, 12-step program, lifestyle changes).",0,B,Y,5952.0,Alcohol Abstinence Therapy - Opioid Receptor Antagonist-Type,N,N,N,N,N,null,null,null,N,N,N
2019.0,2020.0,1.0,65757030001,VIVITROL,NALTREXONE MICROSPHERES,380 MG,"SUSPENSION, EXTENDED RELEASE, RECON (EA)",INTRAMUSC,99.0,87.0,F,ALKERMES INC,"USES: This medication is used to treat alcohol abuse. It is used only in people who have been able to stop drinking for some time before starting treatment with naltrexone. You should not be drinking when you start naltrexone. It can help people drink less alcohol or stop drinking altogether. Naltrexone works in the brain to decrease the desire to drink. It does not work like some other treatments for alcohol abuse (e.g., disulfiram). It will not make you sick when taken with alcohol. This medication is also used to prevent relapse to opioid abuse, after opioid detoxification. It works by blocking the action of opioids. This medication must not be used in people currently taking opiat

## Add Other Elements

### Time Series

In [0]:
# Note AMLODIPINE BESYLATE in ref table as opioid - Can only be removed explicitly by generic name or CDC_OPIOID_LA_SA_ACTING_FLAG in ('SA', 'LA')
# Buprenorphine and buprenorphine HCL not entered - no MME conversion
# https://www.cms.gov/files/document/otp-billing-and-payment-fact-sheet.pdf


global_Opx_conds = "OMS_OPIOID_FLAG = 'Y' and 2020 BETWEEN BEGIN_YR and END_YR AND OMS_NALOXONE_FLAG = 'N' AND OMS_MAT_FLAG = 'N' AND FDB_IND = 'Y' AND FDB_GNN != 'AMLODIPINE BESYLATE'"


flag_dict = {
    'TS_CLMLINES_OTP_NONDRUG': "CASE WHEN SRVC_DLVRD IN ('G2074', 'G2076', 'G2077') THEN 1 ELSE 0 END",
    'TS_CLMLINES_METHADONE_OTP_MAT': "CASE WHEN SRVC_DLVRD IN ('G2067', 'G2078') THEN 1 ELSE 0 END",
    'TS_CLMLINES_METHADONE_INJ_NONOTP': "CASE WHEN SRVC_DLVRD IN ('J1230') THEN 1 ELSE 0 END",
    'TS_CLMLINES_BUPRENORPHINE_ORAL_OTP_MAT': "CASE WHEN SRVC_DLVRD IN ('G2068', 'G2079') THEN 1 ELSE 0 END", 
    'TS_CLMLINES_BUPRENORPHINE_INJ_OTP_MAT': "CASE WHEN SRVC_DLVRD IN ('G2069') THEN 1 ELSE 0 END", 
    'TS_CLMLINES_BUPRENORPHINE_IMPL_OTP_MAT': "CASE WHEN SRVC_DLVRD IN ('G2070', 'G2072') THEN 1 ELSE 0 END""", # G2071 is an implant removal presumed no drug
    'TS_CLMLINES_BUPRENORPHINE_ORAL_NONOTP': "CASE WHEN SRVC_DLVRD IN ('J0571', 'J0572', 'J0573', 'J0574', 'J0575') OR (OMS_MAT_FLAG = 'Y' AND FDB_GNN LIKE '%BUPRENORPHINE%') THEN 1 ELSE 0 END",
    'TS_CLMLINES_BUPRENORPHINE_INJ_NONOTP': "CASE WHEN SRVC_DLVRD IN ('J0592', 'Q9991', 'Q9992') THEN 1 ELSE 0 END",
    
    'TS_CLMLINES_NALTREXONE_INJ_OTP_MAT' : "CASE WHEN SRVC_DLVRD = 'G0273' THEN 1 ELSE 0 END",
    'TS_CLMLINES_NALTREXONE_INJ_NONOTP_MAT' : "CASE WHEN SRVC_DLVRD in ('J1235') THEN 1 ELSE 0 END",
    
    'TS_CLMLINES_NALOXONE' : "CASE WHEN SRVC_DLVRD in ('G2215', 'G2216') OR OMS_NALOXONE_FLAG = 'Y' THEN 1 ELSE 0 END",
    'TS_CLMLINES_OUD_PSYCHOTHERAPY' : "CASE WHEN SRVC_DLVRD IN ('G2086', 'G2087', 'G2088') THEN 1 ELSE 0 END",
    'TS_CUM_OPX_RXERS' : "MAX(cum_distinct_opx_prescribers) OVER (PARTITION BY BENE_ID ORDER BY DATE ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS intermediate",
    'TS_FLAG_DENTIST_OPX_RXERS': f"CASE WHEN PRVDR_SPCLTY = '19' AND {global_Opx_conds} THEN 1 ELSE 0 END",
    'TS_FLAG_PYCHLGST_RXERS': f"CASE WHEN PRVDR_SPCLTY IN ('62', '68') THEN 1 ELSE 0 END",
    'TS_FLAG_NP_PA_OPX_RXERS': f"CASE WHEN PRVDR_SPCLTY IN ('50', '97') AND {global_Opx_conds} THEN 1 ELSE 0 END",
    'TS_FLAG_OPX_ED_VISIT': f"""CASE WHEN
                                      (DGNS_CD1 in ({FULL_Opx_Dx_list}) OR
                                      DGNS_CD2 in ({FULL_Opx_Dx_list}) OR
                                      DGNS_CD3 in ({FULL_Opx_Dx_list}) OR
                                      DGNS_CD4 in ({FULL_Opx_Dx_list}) OR
                                      DGNS_CD5 in ({FULL_Opx_Dx_list}) OR
                                      DGNS_CD6 in ({FULL_Opx_Dx_list}) OR
                                      DGNS_CD7 in ({FULL_Opx_Dx_list}) OR
                                      DGNS_CD8 in ({FULL_Opx_Dx_list}) OR
                                      DGNS_CD9 in ({FULL_Opx_Dx_list}) OR
                                      DGNS_CD10 in ({FULL_Opx_Dx_list}) OR
                                      DGNS_CD11 in ({FULL_Opx_Dx_list}) OR
                                      DGNS_CD12 in ({FULL_Opx_Dx_list}))
                                      THEN 1 
                                    ELSE 0 
                                    END"""
                                    
}

broadcast_dict = { # These results get broadcast to all days from Rx fill date to Rx fill date + days_supply
  
  'TS_MME_ALL_SA' : f"""CASE WHEN (CDC_OPIOID_LA_SA_ACTING_FLAG = 'SA' OR FDB_GNN in ('HYDROCODONE/ACETAMINOPHEN', 'HYDROCODONE/IBUPROFEN', 'OXYCODONE HCL/ACETAMINOPHEN', 'OXYCODONE HCL/ASPIRIN', 'ACETAMINOPHEN WITH CODEINE', 'BUTALBIT/ACETAMIN/CAFF/CODEINE', 'BUTALBITAL/ASPIRIN/CAFFEINE', 'CODEINE/BUTALBITAL/ASA/CAFFEIN')) AND {global_Opx_conds} THEN (SRVC_VOL/CLM_DAYS) * OMS_MME_CONVERSION_FACTOR * OMS_OPIOID_STR ELSE 0 END""",
  
  'TS_MME_ALL_LA' : f"CASE WHEN CDC_OPIOID_LA_SA_ACTING_FLAG = 'LA' AND {global_Opx_conds} THEN (SRVC_VOL/CLM_DAYS) * OMS_MME_CONVERSION_FACTOR * OMS_OPIOID_STR ELSE 0 END",
    
  'TS_DQ_BENZOS' : "CASE WHEN OMS_BENZO_FLAG = 'Y' THEN (SRVC_VOL/CLM_DAYS) END",
  
  'TS_DQ_ANTIPSYCH' : "CASE WHEN ANTIPSYCH_FLAG = 'Y' THEN (SRVC_VOL/CLM_DAYS) END",
  
  'TS_DQ_BUPRENORPHINE_PART_AGON': "CASE WHEN FDB_GNN LIKE '%BUPRENORPHINE%' AND OMS_MAT_FLAG = 'N' THEN (SRVC_VOL/CLM_DAYS) ELSE 0 END",
  'TS_DQ_NALTREXONE_PART_AGON': "CASE WHEN FDB_GNN LIKE '%NALTREXONE%' AND FDB_GNN NOT LIKE '%METHYLNALTREXONE%' THEN (SRVC_VOL/CLM_DAYS) ELSE 0 END"
  }

# output nested array Pandas dataframe
ts_df = (preprocess_timeseries(joined_df, sample=1.0, random_seed = 42)
          .build_time_series(date_col_name = 'first_srvc_dt',
                      start_date = '2020-01-01', 
                      end_date = '2020-10-31',
                      flag_dict = flag_dict,
                      flag_list_max = ['TS_CUM_OPX_RXERS', 'TS_FLAG_DENTIST_OPX_RXERS', 'TS_FLAG_OPX_ED_VISIT','TS_FLAG_PYCHLGST_RXERS', 'TS_FLAG_NP_PA_OPX_RXERS'],
                      broadcast_dict = broadcast_dict)
          .convert_ts_2_array(flag_dict = flag_dict,
                        broadcast_dict = broadcast_dict)
          .output()
         )
         
ts_df.cache().count()

Created bene sample and cached dataframe with distinct benes: 654346
Built long time-series with total rows: 199575530 and 654346 benes
Out[12]: 654346

### Add target

In [0]:
targets = joined_df.select('bene_id', 'target').groupBy('bene_id').agg(max('target').alias('target'))

df_wtargets = ts_df.join(targets, 'bene_id', 'inner')

df_wtargets.count()

Out[13]: 654346

### Add Chronic Conditions

In [0]:
# Assign combined CC table to variable
condition = sqlContext.sql(f"""
SELECT * FROM extracts.mbsf_mbsf_cc_2020 a

FULL OUTER JOIN extracts.mbsf_mbsf_otcc_2020 b
ON a.BENE_ID = b.BENE_ID

"""
              )

In [0]:
# Recoding Conditions to be readable, dummy encoding

cc_dict = {
'ATRIAL_FIB':'Afib'
, 'ALZH_DEMEN':'Alz_Dem'
, 'ALZH': 'Alzheimers' 
, 'AMI':'Act_Myo_Inf'
, 'ANEMIA':'Anemia'
, 'ASTHMA':'Asthma'
, 'CANCER_BREAST':'Breast_Cncr'
, 'CATARACT':'Cataract'
, 'CHF':'CHF'
, 'CHRONICKIDNEY':'CKD'
, 'COPD':'COPD'
, 'CANCER_COLORECTAL':'Colorectal_Cncr'
, 'DEPRESSION':'Depression'
, 'DIABETES':'Diabetes'
, 'CANCER_ENDOMETRIAL':'Endometrial_Cncr'
, 'GLAUCOMA':'Glaucoma'
, 'HIP_FRACTURE':'Hip_Fracture'
, 'HYPERL':'Hyperlipidemia'
, 'HYPERP':'Ben_Prost_Hyprpl'
, 'HYPERT':'Hypertension'
, 'HYPOTH':'Hypothyroidism'
, 'ISCHEMICHEART':'Ischemic_HD'
, 'CANCER_LUNG':'Lung_cncr'
, 'OSTEOPOROSIS':'Osteoporosis'
, 'RA_OA': 'RA_Osteoarth'
, 'STROKE_TIA': 'Stroke'
, 'CANCER_PROSTATE': 'Prostate_Cncr'
, 'ACP_MEDICARE':'Conduct_dis'
, 'ALCO_MEDICARE':'Alcoholism'
, 'ANXI_MEDICARE':'Anxiety'
, 'AUTISM_MEDICARE':'Autism'
, 'BIPL_MEDICARE':'Bipolar'
, 'BRAINJ_MEDICARE':'Brain_injury'
, 'CERPAL_MEDICARE':'Cerebral_palsy'
, 'CYSFIB_MEDICARE':'Cystic_Fibrs'
, 'DEPSN_MEDICARE':'Depressive_dis'
, 'DRUG_MEDICARE':'Drug_use'
, 'EPILEP_MEDICARE':'Epilepsy'
, 'FIBRO_MEDICARE':'Fibromyalgia'
, 'HEARIM_MEDICARE':'Hearing_imp'
, 'HEPVIRAL_MEDICARE':'Hepatitis_viral'
, 'HIVAIDS_MEDICARE':'HIVAIDS'
, 'INTDIS_MEDICARE':'Intillectual_dsblty'
, 'LEADIS_MEDICARE':'Learning_dsblty'
, 'LEUKLYMPH_MEDICARE':'Leuk_lymph'
, 'LIVER_MEDICARE':'Liver_dis'
, 'MIGRAINE_MEDICARE':'Migraine'
, 'MOBIMP_MEDICARE':'Mobility_impairment'
, 'MULSCL_MEDICARE':'Multiple_sclrs'
, 'MUSDYS_MEDICARE':'Muscular_dys'
, 'OBESITY_MEDICARE':'Obesity'
, 'OTHDEL_MEDICARE':'Other_developmental_dis'
, 'OUD_ANY_MEDICARE': 'Opioid_any'
, 'OUD_DX_MEDICARE' : 'Opioid_dx'
, 'OUD_HOSP_MEDICARE' : 'Opioid_hospitalization'  
, 'OUD_MAT_MEDICARE': 'Opioid_medically_assisted'
, 'PSDS_MEDICARE':'Personality_dis'
, 'PTRA_MEDICARE':'PTSD'
, 'PVD_MEDICARE':'Periph_vasc_dis'
, 'SCD_MEDICARE': 'Sickle_cell_disease'
, 'SCHI_MEDICARE':'Schizophrenia'
, 'SCHIOT_MEDICARE':'Schizophrenia_and_other_psychotic'
, 'SPIBIF_MEDICARE':'Spina_bifida'
, 'SPIINJ_MEDICARE':'Spinal_injury'
, 'TOBA_MEDICARE':'Tobacco_use'
, 'ULCERS_MEDICARE':'Ulcers'
, 'VISUAL_MEDICARE':'Visual_imp' 
}

# Create lists of old naming and new naming of chronic conditions
before_list = list(cc_dict.keys())
after_list = list(cc_dict.values())

conditions_recode = condition

# Flag as 1 if claims condition met for CC (1 or 3)
for (x, y) in zip(before_list, after_list):
  conditions_recode=conditions_recode\
  .withColumn(y, F.when((F.col(x) == 1) | (F.col(x) == 3), 1).otherwise(0))

# Add the encode suffix to the columns
for column in after_list:
  conditions_recode=conditions_recode\
.withColumnRenamed(column, "CC_" + column)

after_list = ["{}{}".format('CC_', i) for i in after_list]

after_list.extend(['a.bene_id'])

# Select only human readable column names and add _encode suffix                       
conditions_recode=conditions_recode\
.select(after_list)

# Dropping any reference to Opioid hospitalization from the chronic conditions tables, which would self-reference target
conditions_recode = conditions_recode.drop('CC_Opioid_any', 'CC_Opioid_hospitalization', 'CC_Opioid_dx', 'CC_Opioid_medically_assisted', 'CC_Drug_use')

In [0]:
df_wtargets_cc = df_wtargets.join(conditions_recode, 'bene_id', 'inner')

### Add Demographics

In [0]:
demos = spark.sql("""
SELECT 
a.bene_id,
cast(round(DATEDIFF('2020-12-31', birth_dt)/365.25) AS INT) as STATIC_DEMO_bene_age,
sex_desc as STATIC_DEMO_sex_label,
state_cd as STATIC_DEMO_state_cd,
race_cd_desc as STATIC_DEMO_race_label,
crec_desc as STATIC_DEMO_crec_label,
CASE WHEN ruca.RUCA1 BETWEEN 1 AND 10 THEN ruca.RUCA1 ELSE null END AS STATIC_DEMO_RUCA1,
CAST(first(ADI_NATRANK_BG_2019_v3_1) as INT) as STATIC_DEMO_ADI_NATRANK,
CAST(
CASE
	WHEN CAST(first(ADI_NATRANK_BG_2019_v3_1) AS DECIMAL) Between 0 AND 10.999 THEN '0-10'
	WHEN CAST(first(ADI_NATRANK_BG_2019_v3_1) AS DECIMAL) Between 11 AND 20.999 THEN '11-20'
	WHEN CAST(first(ADI_NATRANK_BG_2019_v3_1) AS DECIMAL) Between 21 AND 30.999 THEN '21-30'
	WHEN CAST(first(ADI_NATRANK_BG_2019_v3_1) AS DECIMAL) Between 31 AND 40.999 THEN '31-40'
	WHEN CAST(first(ADI_NATRANK_BG_2019_v3_1) AS DECIMAL) Between 41 AND 50.999 THEN '41-50'
	WHEN CAST(first(ADI_NATRANK_BG_2019_v3_1) AS DECIMAL) Between 51 AND 60.999 THEN '51-60'
	WHEN CAST(first(ADI_NATRANK_BG_2019_v3_1) AS DECIMAL) Between 61 AND 70.999 THEN '61-70'
	WHEN CAST(first(ADI_NATRANK_BG_2019_v3_1) AS DECIMAL) Between 71 AND 80.999 THEN '71-80'
	WHEN CAST(first(ADI_NATRANK_BG_2019_v3_1) AS DECIMAL) Between 81 AND 90.999 THEN '81-90'
	WHEN CAST(first(ADI_NATRANK_BG_2019_v3_1) AS DECIMAL) Between 91 AND 100 THEN '91-100'
		ELSE 
		CASE
			WHEN first(ADI_NATRANK_BG_2019_v3_1) in ('PH', 'GQ', 'PH-GQ', 'QDI') THEN first(ADI_NATRANK_BG_2019_v3_1)
			ELSE 'Unassigned'
			END 
END AS STRING) AS STATIC_DEMO_ADI_NATRANK_binned

FROM eldb.longitudinal_eldb_bene a

LEFT JOIN eldb.ruca_2010_zipcode ruca
           on a.zip5_ltst = ruca.ZIP_CODE
           and a.month_end = '2020-10-31'
           
LEFT JOIN eldb.adi_ccw_table adi 
          on a.bene_id = adi.bene_id
          and '2020-10-31' between drvd_adr_bgn_dt and drvd_adr_end_dt 

WHERE a.month_end = '2020-10-31'
GROUP BY 1,2,3,4,5,6,7

"""
                 )

In [0]:
df_wtargets_cc_demos = df_wtargets_cc.join(demos, 'bene_id', 'inner')

## Add Nursing Home Data

In [0]:
# Create the array for NH daily status
date_df = spark.sql(f"SELECT sequence(to_date('2020-01-01'), to_date('2020-10-31'), interval 1 day) as date").withColumn("date", explode(col("date")))
NH_daily = (broadcast(date_df.select("date").distinct())
               .crossJoin(joined_df.select("bene_id").distinct())
               .join(spark.sql("SELECT * FROM eldb.mds_timeline_v0_1_0"), ['date', 'bene_id'], "leftouter")
               .select('bene_id', 'date', 'in_nh_today')
               .filter("date BETWEEN '2020-01-01' AND '2020-10-31'")
               .fillna(0)
           )

NH_daily = (NH_daily
                 .groupBy("bene_id")
                 .agg(sort_array(collect_list(struct('date', 'in_nh_today')))  # Ensures proper ordering of array list
                 .alias('collected_list'))
                 .withColumn('TS_IN_NH_TODAY', col(f"collected_list.in_nh_today"))
                 .drop("collected_list")
                ).orderBy('bene_id')

In [0]:
df_wtargets_cc_demos_nh = df_wtargets_cc_demos.join(NH_daily, 'bene_id', 'inner')

## Write table

In [0]:
# Change data types to compress file a bit
df_wtargets_cc_demos_nh = (df_wtargets_cc_demos_nh
 .select([col for col in df_wtargets_cc_demos_nh.columns if col.startswith('TS')==False]+
   [F.col(col).cast(ArrayType(ShortType())).alias(col) for col in df_wtargets_cc_demos_nh.columns if col.startswith('TS_CLMLINES')]+
   [F.col(col).cast(ArrayType(FloatType())).alias(col) for col in df_wtargets_cc_demos_nh.columns if col.startswith('TS_MME')]+
   [F.col(col).cast(ArrayType(ByteType())).alias(col) for col in df_wtargets_cc_demos_nh.columns if col.startswith('TS_FLAG')]+
   [F.col(col).cast(ArrayType(ShortType())).alias(col) for col in df_wtargets_cc_demos_nh.columns if col.startswith('TS_CUM')]+
   [F.col(col).cast(ArrayType(ShortType())).alias(col) for col in df_wtargets_cc_demos_nh.columns if col.startswith('TS_DQ')]
 )
           )

In [0]:
# Extra cleanup - drop unnecessary and imputation

median_ruca = df_wtargets_cc_demos_nh.groupBy().agg(F.percentile_approx('STATIC_DEMO_RUCA1', 0.5)).collect()[0][0]
df_wtargets_cc_demos_nh = df_wtargets_cc_demos_nh.drop('STATIC_DEMO_ADI_NATRANK','STATIC_DEMO_ADI_STATERNK_str')
df_wtargets_cc_demos_nh = df_wtargets_cc_demos_nh.fillna({"STATIC_DEMO_state_cd":"Unassigned", 'STATIC_DEMO_RUCA1': median_ruca}, subset=['STATIC_DEMO_state_cd', 'STATIC_DEMO_RUCA1'])

In [0]:
df_wtargets_cc_demos_nh.write \
  .format("delta")\
  .option("overwriteSchema", "true")\
  .mode("overwrite") \
  .saveAsTable(f"eldb.opioid_SA_LA_hosp_sktime_table")         
         

BEGIN_YR,END_YR,YEAR_GAP,FDB_NDC,FDB_BN,FDB_GNN,FDB_STR,FDB_FORM,FDB_RT,FDB_GTC,FDB_TC,FDB_CL,FDB_MFG,FDB_MONO,FDB_DEA,FDB_BG_IND,FDB_IND,FDB_ETC_ID,FDB_ETC_NAME,OMS_OPIOID_FLAG,OMS_MAT_FLAG,OMS_METHADONE_FLAG,OMS_BENZO_FLAG,OMS_NALOXONE_FLAG,OMS_MME_CONVERSION_FACTOR,OMS_OPIOID_STR,CDC_OPIOID_LA_SA_ACTING_FLAG,OTC_MDCD,ANTIBIOTIC_FLAG,ANTIPSYCH_FLAG
2016.0,2020.0,1.0,54868079202,HYDROCHLOROTHIAZIDE,HYDROCHLOROTHIAZIDE,25 MG,TABLET,ORAL,56.0,79.0,F,PHYSICIANS TC.,"USES: This medication is used to treat high blood pressure. Lowering high blood pressure helps prevent strokes, heart attacks, and kidney problems. Hydrochlorothiazide belongs to a class of drugs known as diuretics/""water pills."" It works by causing you to make more urine. This helps your body get rid of extra salt and water. This medication also reduces extra fluid in the body (edema) caused by conditions such as heart failure, liver disease, or kidney disease. This can lessen symptoms such as shortness of breath or swelling in your ankles or feet.",0,G,Y,254.0,Diuretic - Thiazides and Related,N,N,N,N,N,null,null,null,N,N,N
2015.0,2020.0,1.0,54868094301,METROGEL,METRONIDAZOLE,0.75 %,GEL (GRAM),TOPICAL,86.0,95.0,F,PHYSICIANS TC.,"USES: This medication is used on the skin to treat a certain skin disorder known as rosacea, a type of adult acne. It may help to decrease redness, swelling and the number of pimples caused by rosacea. This medication is an antibiotic. For the treatment of rosacea, metronidazole is believed to work by decreasing swelling (inflammation). Some brands of this medication also contain sunscreens (see also Notes section).",0,B,Y,5717.0,"Dermatological - Rosacea Therapy, Topical",N,N,N,N,N,null,null,null,N,N,N
2013.0,2014.0,1.0,54868094301,METROGEL,METRONIDAZOLE,0.75 %,GEL (GRAM),TOPICAL,86.0,95.0,F,PHYSICIANS TC.,"USES: This medication is used on the skin to treat a certain skin disorder known as rosacea, a type of adult acne. It may help to decrease redness, swelling and the number of pimples caused by rosacea. This medication is an antibiotic. For the treatment of rosacea, metronidazole is believed to work by decreasing swelling (inflammation). Some brands of this medication also contain sunscreens (see also Notes section).",0,B,Y,938.0,Acne Therapy Topical - Anti-infective,N,N,N,N,N,null,null,null,N,N,N
2015.0,2020.0,1.0,54868094300,METROGEL,METRONIDAZOLE,0.75 %,GEL (GRAM),TOPICAL,86.0,95.0,F,PHYSICIANS TC.,"USES: This medication is used on the skin to treat a certain skin disorder known as rosacea, a type of adult acne. It may help to decrease redness, swelling and the number of pimples caused by rosacea. This medication is an antibiotic. For the treatment of rosacea, metronidazole is believed to work by decreasing swelling (inflammation). Some brands of this medication also contain sunscreens (see also Notes section).",0,O,Y,5717.0,"Dermatological - Rosacea Therapy, Topical",N,N,N,N,N,null,null,null,N,N,N
2013.0,2014.0,1.0,54868094300,METROGEL,METRONIDAZOLE,0.75 %,GEL (GRAM),TOPICAL,86.0,95.0,F,PHYSICIANS TC.,"USES: This medication is used on the skin to treat a certain skin disorder known as rosacea, a type of adult acne. It may help to decrease redness, swelling and the number of pimples caused by rosacea. This medication is an antibiotic. For the treatment of rosacea, metronidazole is believed to work by decreasing swelling (inflammation). Some brands of this medication also contain sunscreens (see also Notes section).",0,O,Y,938.0,Acne Therapy Topical - Anti-infective,N,N,N,N,N,null,null,null,N,N,N
2016.0,2020.0,1.0,54868094006,NIZORAL,KETOCONAZOLE,200 MG,TABLET,ORAL,22.0,94.0,F,PHYSICIANS TC.,USES: See also Warning section. Ketoconazole is used to treat certain serious fungal infections in the body. Ketoconazole belongs to the class of drugs called azole antifungals. It works by stopping the growth of the fungus. Ketoconazole should not be used to treat fungal infections on the skin and nails due to the risk of serious side effects and

BEGIN_YR,END_YR,YEAR_GAP,FDB_NDC,FDB_BN,FDB_GNN,FDB_STR,FDB_FORM,FDB_RT,FDB_GTC,FDB_TC,FDB_CL,FDB_MFG,FDB_MONO,FDB_DEA,FDB_BG_IND,FDB_IND,FDB_ETC_ID,FDB_ETC_NAME,OMS_OPIOID_FLAG,OMS_MAT_FLAG,OMS_METHADONE_FLAG,OMS_BENZO_FLAG,OMS_NALOXONE_FLAG,OMS_MME_CONVERSION_FACTOR,OMS_OPIOID_STR,CDC_OPIOID_LA_SA_ACTING_FLAG,OTC_MDCD,ANTIBIOTIC_FLAG,ANTIPSYCH_FLAG
2013.0,2020.0,1.0,57866707601,AMLODIPINE BESYLATE,AMLODIPINE BESYLATE,5 MG,TABLET,ORAL,38.0,76.0,F,VIBRANTA INC.,"USES: Amlodipine is used with or without other medications to treat high blood pressure. Lowering high blood pressure helps prevent strokes, heart attacks, and kidney problems. Amlodipine belongs to a class of drugs known as calcium channel blockers. It works by relaxing blood vessels so blood can flow more easily. Amlodipine is also used to prevent certain types of chest pain (angina). It may help to increase your ability to exercise and decrease the frequency of angina attacks. It should not be used to treat attacks of chest pain when they occur. Use other medications (such as sublingual nitroglycerin) to relieve attacks of chest pain as directed by your doctor.",0,O,Y,4609.0,Calcium Channel Blockers - Dihydropyridines,Y,N,N,N,N,1.5,10.0,null,N,N,N


BEGIN_YR,END_YR,YEAR_GAP,FDB_NDC,FDB_BN,FDB_GNN,FDB_STR,FDB_FORM,FDB_RT,FDB_GTC,FDB_TC,FDB_CL,FDB_MFG,FDB_MONO,FDB_DEA,FDB_BG_IND,FDB_IND,FDB_ETC_ID,FDB_ETC_NAME,OMS_OPIOID_FLAG,OMS_MAT_FLAG,OMS_METHADONE_FLAG,OMS_BENZO_FLAG,OMS_NALOXONE_FLAG,OMS_MME_CONVERSION_FACTOR,OMS_OPIOID_STR,CDC_OPIOID_LA_SA_ACTING_FLAG,OTC_MDCD,ANTIBIOTIC_FLAG,ANTIPSYCH_FLAG
2020.0,2020.0,1.0,70651000203,PENTETATE ZINC TRISODIUM,PENTETATE ZINC TRISODIUM,200 MG/ML,AMPUL (ML),INTRAVEN,99.0,54.0,F,HAMELN-GSMS,null,0,B,Y,6135.0,Chelating Agents - Others,N,N,N,N,N,null,null,null,N,N,N
2019.0,2020.0,1.0,70710113101,CHLORPROMAZINE HCL,CHLORPROMAZINE HCL,50 MG,TABLET,ORAL,80.0,7.0,F,ZYDUS PHARMACEU,"USES: This medication is used to treat certain mental/mood disorders (such as schizophrenia, psychotic disorders, manic phase of bipolar disorder, severe behavioral problems in children). Chlorpromazine helps you to think more clearly, feel less nervous, and take part in everyday life. It can reduce aggressive behavior and the desire to hurt yourself/others. It may also help to decrease hallucinations (hearing/seeing things that are not there). Chlorpromazine is a psychiatric medication that belongs to the class of drugs called phenothiazine antipsychotics. It works by helping to restore the balance of certain natural substances in the brain. Chlorpromazine is also used to control nausea/vomiting, relieve prolonged hiccups, relieve restlessness/anxiety before surgery, and help treat tetanus.",0,G,Y,2596.0,"Antipsychotic - Phenothiazines, Aliphatic",N,N,N,N,N,null,null,null,N,N,Y
2019.0,2020.0,1.0,70710113001,CHLORPROMAZINE HCL,CHLORPROMAZINE HCL,25 MG,TABLET,ORAL,80.0,7.0,F,ZYDUS PHARMACEU,"USES: This medication is used to treat certain mental/mood disorders (such as schizophrenia, psychotic disorders, manic phase of bipolar disorder, severe behavioral problems in children). Chlorpromazine helps you to think more clearly, feel less nervous, and take part in everyday life. It can reduce aggressive behavior and the desire to hurt yourself/others. It may also help to decrease hallucinations (hearing/seeing things that are not there). Chlorpromazine is a psychiatric medication that belongs to the class of drugs called phenothiazine antipsychotics. It works by helping to restore the balance of certain natural substances in the brain. Chlorpromazine is also used to control nausea/vomiting, relieve prolonged hiccups, relieve restlessness/anxiety before surgery, and help treat tetanus.",0,G,Y,2596.0,"Antipsychotic - Phenothiazines, Aliphatic",N,N,N,N,N,null,null,null,N,N,Y
2019.0,2020.0,1.0,70710112901,CHLORPROMAZINE HCL,CHLORPROMAZINE HCL,10 MG,TABLET,ORAL,80.0,7.0,F,ZYDUS PHARMACEU,"USES: This medication is used to treat certain mental/mood disorders (such as schizophrenia, psychotic disorders, manic phase of bipolar disorder, severe behavioral problems in children). Chlorpromazine helps you to think more clearly, feel less nervous, and take part in everyday life. It can reduce aggressive behavior and the desire to hurt yourself/others. It may also help to decrease hallucinations (hearing/seeing things that are not there). Chlorpromazine is a psychiatric medication that belongs to the class of drugs called phenothiazine antipsychotics. It works by helping to restore the balance of certain natural substances in the brain. Chlorpromazine is also used to control nausea/vomiting, relieve prolonged hiccups, relieve restlessness/anxiety before surgery, and help treat tetanus.",0,G,Y,2596.0,"Antipsychotic - Phenothiazines, Aliphatic",N,N,N,N,N,null,null,null,N,N,Y
2019.0,2020.0,1.0,70710112801,URSODIOL,URSODIOL,500 MG,TABLET,ORAL,65.0,5.0,F,ZYDUS PHARMACEU,"USES: Ursodiol is used to dissolve certain types of gallstones, to prevent gallstones from forming in obese patients who are losing weight rapidly, and to treat a certain type of liver disease (primary biliary cholangitis). Ursodiol is a bile acid.",0,G,Y,3012.0,Gallstone Solubilizing (Litholysis) Agents,N,N,N,N,N,null,null

FDB_GNN,CDC_OPIOID_LA_SA_ACTING_FLAG,OMS_OPIOID_STR,OMS_MME_CONVERSION_FACTOR,count
PENTAZOCINE HCL/NALOXONE HCL,null,50.0,0.37,2
BUPRENORPHINE HCL,LA,900.0,null,2
MORPHINE SULFATE,SA,15.0,1.0,40
TRAMADOL HCL,null,50.0,0.1,196
OXYCODONE HCL,LA,20.0,1.5,65
TAPENTADOL HCL,SA,50.0,0.4,15
BUPRENORPHINE,LA,5.0,null,9
FENTANYL,SA,400.0,0.18,2
MORPHINE SULFATE,LA,40.0,1.0,4
BUTORPHANOL TARTRATE,SA,10.0,7.0,9


BEGIN_YR,END_YR,YEAR_GAP,FDB_NDC,FDB_BN,FDB_GNN,FDB_STR,FDB_FORM,FDB_RT,FDB_GTC,FDB_TC,FDB_CL,FDB_MFG,FDB_MONO,FDB_DEA,FDB_BG_IND,FDB_IND,FDB_ETC_ID,FDB_ETC_NAME,OMS_OPIOID_FLAG,OMS_MAT_FLAG,OMS_METHADONE_FLAG,OMS_BENZO_FLAG,OMS_NALOXONE_FLAG,OMS_MME_CONVERSION_FACTOR,OMS_OPIOID_STR,CDC_OPIOID_LA_SA_ACTING_FLAG,OTC_MDCD,ANTIBIOTIC_FLAG,ANTIPSYCH_FLAG
2020.0,2020.0,0.0,16729055010,BUPRENORPHINE-NALOXONE,BUPRENORPHINE HCL/NALOXONE HCL,8 MG-2 MG,"TABLET, SUBLINGUAL",SUBLINGUAL,99.0,99.0,F,ACCORD HEALTHCA,"USES: This medication contains 2 medicines: buprenorphine and naloxone. It is used to treat opioid dependence/addiction. Buprenorphine belongs to a class of drugs called mixed opioid agonist-antagonists. Buprenorphine helps prevent withdrawal symptoms caused by stopping other opioids. Naloxone is an opioid antagonist that blocks the effect of opioids and can cause severe opioid withdrawal when injected. It has little effect when taken by mouth or dissolved under the tongue. It is combined with buprenorphine to prevent abuse and misuse (injection) of this medication. This combination medication is used as part of a complete treatment program for drug abuse (such as compliance monitoring, counseling, behavioral contract, lifestyle changes).",3,G,Y,571.0,"Agents for Opioid Withdrawal, Opioid-Type",N,Y,N,N,N,null,null,null,N,N,N
2020.0,2020.0,0.0,16729054910,BUPRENORPHINE-NALOXONE,BUPRENORPHINE HCL/NALOXONE HCL,2 MG-0.5MG,"TABLET, SUBLINGUAL",SUBLINGUAL,99.0,99.0,F,ACCORD HEALTHCA,"USES: This medication contains 2 medicines: buprenorphine and naloxone. It is used to treat opioid dependence/addiction. Buprenorphine belongs to a class of drugs called mixed opioid agonist-antagonists. Buprenorphine helps prevent withdrawal symptoms caused by stopping other opioids. Naloxone is an opioid antagonist that blocks the effect of opioids and can cause severe opioid withdrawal when injected. It has little effect when taken by mouth or dissolved under the tongue. It is combined with buprenorphine to prevent abuse and misuse (injection) of this medication. This combination medication is used as part of a complete treatment program for drug abuse (such as compliance monitoring, counseling, behavioral contract, lifestyle changes).",3,G,Y,571.0,"Agents for Opioid Withdrawal, Opioid-Type",N,Y,N,N,N,null,null,null,N,N,N
2020.0,2020.0,0.0,71335129601,BUPRENORPHINE-NALOXONE,BUPRENORPHINE HCL/NALOXONE HCL,2 MG-0.5MG,"TABLET, SUBLINGUAL",SUBLINGUAL,99.0,99.0,F,BRYANT RANCH PR,"USES: This medication contains 2 medicines: buprenorphine and naloxone. It is used to treat opioid dependence/addiction. Buprenorphine belongs to a class of drugs called mixed opioid agonist-antagonists. Buprenorphine helps prevent withdrawal symptoms caused by stopping other opioids. Naloxone is an opioid antagonist that blocks the effect of opioids and can cause severe opioid withdrawal when injected. It has little effect when taken by mouth or dissolved under the tongue. It is combined with buprenorphine to prevent abuse and misuse (injection) of this medication. This combination medication is used as part of a complete treatment program for drug abuse (such as compliance monitoring, counseling, behavioral contract, lifestyle changes).",3,G,Y,571.0,"Agents for Opioid Withdrawal, Opioid-Type",N,Y,N,N,N,null,null,null,N,N,N
2020.0,2020.0,0.0,71335116303,BUPRENORPHINE HCL,BUPRENORPHINE HCL,8 MG,"TABLET, SUBLINGUAL",SUBLINGUAL,99.0,99.0,F,BRYANT RANCH PR,"USES: Buprenorphine is used to treat dependence/addiction to opioids. Buprenorphine belongs to a class of drugs called mixed opioid agonist-antagonists. It helps prevent withdrawal symptoms caused by stopping other opioids. It is used as part of a complete treatment program for drug abuse (such as compliance monitoring, counseling, behavioral contract, lifestyle changes).",3,G,Y,571.0,"Agents for Opioid Withdrawal, Opioid-Type",N,Y,N,N,N,null,null,null,N,N,N
2020.0,2020.0,0.0,71335116302,BUPRENORPHINE HCL,BUPRENORPHINE HCL,8 MG,"TA

...

Out[12]: '\nopioid_df = (ptd_df.join(df, "FDB_NDC", \'inner\')\n .filter("OMS_OPIOID_FLAG = \'Y\' and 2020 BETWEEN BEGIN_YR and END_YR AND OMS_NALOXONE_FLAG = \'N\'")\n .withColumn("Total_MME_calculated", joined_df.QTY_DSPNSD_NUM * joined_df.OMS_OPIOID_STR * joined_df.OMS_MME_CONVERSION_FACTOR)\n )\n'

Out[20]: '\njoined_df = (ptd_df.join(df, "FDB_NDC", \'inner\')\n .filter("OMS_OPIOID_FLAG = \'Y\' and 2020 BETWEEN BEGIN_YR and END_YR AND OMS_NALOXONE_FLAG = \'N\'")\n )\njoined_df = joined_df.withColumn("Total_MME_calculated", joined_df.QTY_DSPNSD_NUM * df.OMS_OPIOID_STR * df.OMS_MME_CONVERSION_FACTOR) \n\ndisplay(joined_df)\n'

Out[21]: '\nopioid_df = (ptd_df.join(df, "FDB_NDC", \'inner\')\n .filter("OMS_OPIOID_FLAG = \'Y\' and 2020 BETWEEN BEGIN_YR and END_YR AND OMS_NALOXONE_FLAG = \'N\'")\n .withColumn("Total_MME_calculated", joined_df.QTY_DSPNSD_NUM * joined_df.OMS_OPIOID_STR * joined_df.OMS_MME_CONVERSION_FACTOR)\n )\n'

Out[13]: '\nfrom pyspark.sql.functions import mean, count, col, isnull, when\n\ndisplay(joined_df.groupBy("FDB_GNN", "CDC_OPIOID_LA_SA_ACTING_FLAG").agg(count(when(isnull(\'Total_MME_calculated\'), 1)), count(\'FDB_NDC\'))\n )\n'

Out[22]: '\nfrom pyspark.sql.functions import mean, count, col, isnull, when\n\ndisplay(joined_df.groupBy("FDB_GNN", "CDC_OPIOID_LA_SA_ACTING_FLAG").agg(count(when(isnull(\'Total_MME_calculated\'), 1)), count(\'FDB_NDC\'))\n )\n'